In [8]:
import json
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import UnstructuredHTMLLoader
import xml.etree.ElementTree as ET
import os

manifest = []

# Define function to fetch and parse each sitemap page
def process_sitemap_page(sitemap_url):
    # fetch the XML sitemap
    response = requests.get(sitemap_url)

    # parse the XML content
    root = ET.fromstring(response.content)


    # iterate over all URLs in the sitemap
    for url in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
        # fetch the HTML content
        response = requests.get(url.text)

        # convert the HTML content to a BeautifulSoup object
        soup = BeautifulSoup(response.text, 'html.parser')

        article_content = soup.find('article')

        # save the HTML content to a local file
        filename = 'html/' + url.text.replace('https://', '').replace('/', '_') + '.html'
        
        with open(filename, "w", encoding='utf-8') as f:
            f.write(str(article_content.prettify()))

        manifest.append({"filename": filename, "url": url.text})


# Call the function for each sitemap page
process_sitemap_page("https://www.fairwork.gov.au/sitemap.xml?page=1")
process_sitemap_page("https://www.fairwork.gov.au/sitemap.xml?page=2")

with open('manifest_dump.json', 'w') as f:
    f.write(json.dumps(manifest))

